# Piperack Segregator 1.0

> Segregate different elements and apply profiles as required with integrated profile applicator

In [1]:
from base import *
from base.piperack.load import *

In [2]:
# Object creation with provided data
PR = Piperack('MPR_C-C')
PR = PR.load_from_pickle(filename=fr'piperack_objects\{PR.name}.pkl')

PR_S = PR.generate_structural_components(geometry=geometry,property=property)

display(HTML(dark_theme_css))
display(widgets.HBox([
    create_button_bar_for_dict(heading='Columns', item_dict=PR_S.columns_y, selector=select_beams),
    create_button_bar_for_dict(heading='Stub Columns', item_dict=PR_S.stub_columns_y, selector=select_beams),
    create_button_bar_for_dict(heading='Tier Beams', item_dict={tier.base.y: tier.beams for tier in PR.tiers}, selector=select_beams),
    create_button_bar_for_dict(heading='Tier Int Beams', item_dict={tier.base.y: tier.int_beams for tier in PR.tiers}, selector=select_beams),
    create_button_bar_for_dict(heading='VB', item_dict=group_beams_by_y(PR_S.vertical_braces), selector=select_beams),
    create_button_bar_for_dict(heading='PB', item_dict=group_beams_by_y(PR_S.plan_braces), selector=select_beams),
    create_button_bar_for_dict(heading='Long Beams', item_dict=group_beams_by_y(PR_S.long_beams), selector=select_beams),
    create_button_bar_for_dict(heading='Long Int Beams', item_dict=group_beams_by_y(PR_S.int_long_beams), selector=select_beams)
], layout=dark_hbox))

staad_profiles = get_staad_profiles()
staad_section_ref_nos = {profile['name']:profile for profile in staad_profiles}

staad_profiles_widget = display_data_widget(staad_profiles)
steel_widget, steel_dropdown = create_steel_section_widget(steel_sections)

profile_button_bar = widgets.VBox([create_button(label='Insert Profile',predicate=lambda:insert_profile_button_click(get_section_ref_no=get_section_ref_no,
                                                                                                                     steel_dropdown=steel_dropdown,
                                                                                                                     staad_section_ref_nos=staad_section_ref_nos,
                                                                                                                     )), 
                                  
                                   create_button(label='Apply Profile',predicate=lambda:apply_profile_button_click(get_section_ref_no=get_section_ref_no,
                                                                                                                   steel_dropdown=steel_dropdown,
                                                                                                                   staad_section_ref_nos=staad_section_ref_nos,
                                                                                                                   geometry=geometry,get_selected_beam_nos=get_selected_beam_nos,
                                                                                                                   beam_list_copy_and_display=beam_list_copy_and_display,
                                                                                                                   assign_profile=assign_profile))])

display(widgets.HBox([staad_profiles_widget,steel_widget, profile_button_bar]))